### Creating `model.tar.gz`

In [2]:
import tarfile

model_path = "./inference/lit-bert-for-maskedlm-230126.pth"
code_path = "./inference/code/"

zipped_model_path = "./model.tar.gz"

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add(model_path, arcname="lit-bert-for-maskedlm-230126.pth")
    tar.add(code_path, arcname="code/")

### Deploy (Real-time Inference)

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import time

endpoint_name = "lit-bert-for-maskedlm-yymmdd-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Set the 'role' manually if operate on local environment.
model = PyTorchModel(
    entry_point="inference.py",
    model_data=zipped_model_path,
    role=get_execution_role(),
    framework_version="1.5",
    py_version="py3",
)

predictor = model.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name
)

### Deploy (Serverless Inference)

Reference:
* [Deploying ML models using SageMaker Serverless Inference](https://aws.amazon.com/blogs/machine-learning/deploying-ml-models-using-sagemaker-serverless-inference-preview/)
* [Docker Registry Paths and Example Code for Asia Pacific(`image_uri`)](https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-ap-northeast-1.html#pytorch-ap-northeast-1.title)
* [Available Deep Learning Containers Images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)
* [About create-endpoint-config](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/sagemaker/create-endpoint-config.html)

In [ ]:
# Creating Model
import boto3
import sagemaker

from time import gmtime, strftime

region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)

#Role to give SageMaker permission to access AWS services.
sagemaker_role = sagemaker.get_execution_role()

bucket_name = '<your bucket_name>'
s3_prefix = '<your s3_prefix>'

#Get model from S3
model_url = f"s3://{bucket_name}/{s3_prefix}/model.tar.gz".format(bucket_name=bucket_name, s3_prefix=s3_prefix)

#Get container image (prebuilt example)
from sagemaker import image_uris
image_uri = image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.12.0",
    py_version="py38",
    image_scope='inference',
    instance_type="ml.t2.medium"
)

#Create model
model_name = "mlbert-serverless-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    Containers = [{
        "Image": image_uri,
        "Mode": "SingleModel",
        "ModelDataUrl": model_url,
    }]
)

In [ ]:
# Creating Endpoint Configurations.
mlbert_serverless_epc_name = "mlbert-serverless-epc-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=mlbert_serverless_epc_name,
    ProductionVariants=[
        {
        "VariantName": "mlVariant",
        "ModelName": model_name,
        "ServerlessConfig": {
            "MemorySizeInMB": 2048,
            "MaxConcurrency": 1,
            },
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

In [ ]:
# Deploy
endpoint_name = "mlbert-serverless-ep-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=mlbert_serverless_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(30)

describe_endpoint_response

### Make a Prediction

#### SageMaker Studio

In [ ]:
import json
import sagemaker

sm = sagemaker.Session().sagemaker_runtime_client

payload ="明日の天気はどうですか。"

response = sm.invoke_endpoint(
    EndpointName=endpoint_name, 
    Body=payload, 
    ContentType="application/json"
)

result = json.loads(response['Body'].read().decode('utf8'))
result

#### Locally

In [ ]:
import boto3
import ujson as json

client = boto3.client('sagemaker-runtime', region_name='ap-northeast-1')

payload ="今日の天気はどうですか。"

response = client.invoke_endpoint(
    EndpointName=endpoint_name, Body=payload, ContentType="text/csv"
)

resp = json.loads(response['Body'].read().decode('utf-8'))
resp

### Test API Gateway

In [ ]:
import requests
import json

# Set throtting at 'Stage -> Stage Editor'

api_id = '<API Gateway ID>'

url = f"https://{api_id}.execute-api.ap-northeast-1.amazonaws.com/api/bertjsc".format(api_id=api_id)

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
payload = {'text': '今日はいい天気です。'}

resp = requests.post(url, data =json.dumps(payload), headers=headers)

resp.json()